## Settings

In [1]:
import sys
sys.path.append('..')

In [2]:
import os
import random
from scipy import stats
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from inputs.funcs import load_data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [3]:
os.listdir('../inputs/')

['test.csv',
 '__init__.py',
 '__pycache__',
 'funcs.py',
 'train.csv',
 'gender_submission.csv']

In [4]:
train_x, train_y, test_x = load_data()

In [5]:
print('train_x: ', train_x.columns.values)
print('train_y: ', test_x.columns.values)

train_x:  ['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare'
 'Cabin' 'Embarked']
train_y:  ['PassengerId' 'Pclass' 'Name' 'Sex' 'Age' 'SibSp' 'Parch' 'Ticket' 'Fare'
 'Cabin' 'Embarked']


In [6]:
train_x.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [7]:
print('Col to drop:')
print(['PassengeId'])

Col to drop:
['PassengeId']


In [8]:
print('Cols to normalize:')
print(['Age', 'Fare'])

Cols to normalize:
['Age', 'Fare']


In [46]:
print('Cols to one hot encode:')
print(train_x.dtypes[~train_x.dtypes.isin([int, float])].index.to_list())

Cols to one hot encode:
['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']


## Cols to nomalize

In [10]:
print('Cols to normalize:')
print(['Age', 'Fare'])

Cols to normalize:
['Age', 'Fare']


### Age 

In [48]:
train_x, train_y, test_x = load_data()

In [54]:
print('Col has na in train_x: ', train_x['Age'].isna().sum(), '/', len(train_x))
print('Col has na in test_x: ', test_x['Age'].isna().sum(), '/', len(test_x))

Col has na in train_x:  177 / 891
Col has na in test_x:  86 / 418


#### Fill na by target encoding

In [13]:
data_tmp = pd.DataFrame({'Age': train_x['Age'], 'Survived': train_y})
target_mean = data_tmp.groupby('Survived').mean().to_dict()
index_nan_0 = (train_x['Age'].isna()) & (train_y == 0)
index_nan_1 = (train_x['Age'].isna()) & (train_y == 1)
train_x.loc[index_nan_0, 'Age'] = target_mean['Age'][0]
train_x.loc[index_nan_1, 'Age'] = target_mean['Age'][1]

In [14]:
random.seed(0)
index_nan = test_x['Age'].isna()
test_x['Age'] = test_x['Age'].apply(lambda x:
                                         random.choice([target_mean['Age'][0], target_mean['Age'][1]]) if np.isnan(x) else x
                                    )

In [15]:
scaler = StandardScaler()
train_x['Age'] = scaler.fit_transform(train_x['Age'].values.reshape(len(train_x), -1)).ravel()
test_x['Age'] = scaler.transform(test_x['Age'].values.reshape(len(test_x), -1)).ravel()

In [16]:
train_x[['PassengerId', 'Age']].to_feather('../features/nn/age_train_std_te.feather')
test_x[['PassengerId', 'Age']].to_feather('../features/nn/age_test_std_te.feather')

#### Fill na with average

In [17]:
train_x, train_y, test_x = load_data()

In [18]:
all_x = pd.concat([train_x, test_x])
mean = train_x['Age'].mean()

In [19]:
index_nan = train_x['Age'].isna()
train_x.loc[index_nan, 'Age'] = mean

In [20]:
index_nan = test_x['Age'].isna()
test_x.loc[index_nan, 'Age'] = mean

In [21]:
scaler = StandardScaler()
train_x['Age'] = scaler.fit_transform(train_x['Age'].values.reshape(len(train_x), -1)).ravel()
test_x['Age'] = scaler.transform(test_x['Age'].values.reshape(len(test_x), -1)).ravel()

In [22]:
train_x[['PassengerId', 'Age']].to_feather('../features/nn/age_train_std_mean.feather')
test_x[['PassengerId', 'Age']].to_feather('../features/nn/age_test_std_mean.feather')

#### Is under 10

In [55]:
train_x, train_y, test_x = load_data()

In [56]:
train_x['Age_<10'] = train_x['Age'].apply(lambda x: 0 if x < 10 else 1)
test_x['Age_<10'] = test_x['Age'].apply(lambda x: 0 if x < 10 else 1)

In [57]:
train_x[['PassengerId', 'Age_<10']].to_feather('../features/nn/age_train_<10.feather')
test_x[['PassengerId', 'Age_<10']].to_feather('../features/nn/age_test_<10.feather')

#### In 30s

In [59]:
train_x, train_y, test_x = load_data()

In [60]:
train_x['Age_30s'] = train_x['Age'].apply(lambda x: 0 if x > 30 or x < 30 else 1)
test_x['Age_30s'] = test_x['Age'].apply(lambda x: 0 if x > 30 or x < 30 else 1)

In [61]:
train_x[['PassengerId', 'Age_30s']].to_feather('../features/nn/age_train_30s.feather')
test_x[['PassengerId', 'Age_30s']].to_feather('../features/nn/age_test_30s.feather')

### Fare

In [23]:
train_x, train_y, test_x = load_data()

In [24]:
print('Col has na in train_x: ', train_x['Fare'].isna().any())
print('Col has na in test_x: ', test_x['Fare'].isna().any())

Col has na in train_x:  False
Col has na in test_x:  True


#### Fill na by target encoding

In [25]:
data_tmp = pd.DataFrame({'Fare': train_x['Fare'], 'Survived': train_y})
target_mean = data_tmp.groupby('Survived').mean().to_dict()
index_nan_0 = (train_x['Fare'].isna()) & (train_y == 0)
index_nan_1 = (train_x['Fare'].isna()) & (train_y == 1)
train_x.loc[index_nan_0, 'Fare'] = target_mean['Fare'][0]
train_x.loc[index_nan_1, 'Fare'] = target_mean['Fare'][1]

In [26]:
random.seed(0)
index_nan = test_x['Fare'].isna()
test_x['Fare'] = test_x['Fare'].apply(lambda x:
                                         random.choice([target_mean['Fare'][0], target_mean['Fare'][1]]) if np.isnan(x) else x
                                     )

In [27]:
scaler = StandardScaler()
train_x['Fare'] = scaler.fit_transform(train_x['Fare'].values.reshape(len(train_x), -1)).ravel()
test_x['Fare'] = scaler.transform(test_x['Fare'].values.reshape(len(test_x), -1)).ravel()

In [28]:
train_x[['PassengerId', 'Fare']].to_feather('../features/nn/fare_train_std_te.feather')
test_x[['PassengerId', 'Fare']].to_feather('../features/nn/fare_test_std_te.feather')

#### Fill na with average

In [29]:
train_x, train_y, test_x = load_data()

In [30]:
all_x = pd.concat([train_x, test_x])
mean = all_x['Fare'].mean()

In [31]:
index_nan = test_x['Fare'].isna()
test_x.loc[index_nan, 'Fare'] = mean

In [32]:
scaler = StandardScaler()
train_x['Fare'] = scaler.fit_transform(train_x['Fare'].values.reshape(len(train_x), -1)).ravel()
test_x['Fare'] = scaler.transform(test_x['Fare'].values.reshape(len(test_x), -1)).ravel()

In [33]:
train_x[['PassengerId', 'Fare']].to_feather('../features/nn/fare_train_std_mean.feather')
test_x[['PassengerId', 'Fare']].to_feather('../features/nn/fare_test_std_mean.feather')

#### Under 10

In [67]:
train_x, train_y, test_x = load_data()

In [68]:
train_x['Fare_<10'] = train_x['Fare'].apply(lambda x: 0 if x > 10 else 1)
test_x['Fare_<10'] = test_x['Fare'].apply(lambda x: 0 if x > 10 else 1)

In [69]:
train_x[['PassengerId', 'Fare_<10']].to_feather('../features/nn/fare_train_<10.feather')
test_x[['PassengerId', 'Fare_<10']].to_feather('../features/nn/fare_test_<10.feather')

#### Over 50

In [63]:
train_x, train_y, test_x = load_data()

In [64]:
train_x['Fare_>50'] = train_x['Fare'].apply(lambda x: 0 if x < 50 else 1)
test_x['Fare_>50'] = test_x['Fare'].apply(lambda x: 0 if x < 50 else 1)

In [66]:
train_x[['PassengerId', 'Fare_>50']].to_feather('../features/nn/fare_train_>50.feather')
test_x[['PassengerId', 'Fare_>50']].to_feather('../features/nn/fare_test_>50.feather')

## Cols to one hot encode

In [34]:
train_x, train_y, test_x = load_data()
cat_cols = ['Pclass'] + train_x.dtypes[
                ~train_x.dtypes.isin([np.dtype('int64'), np.dtype('float64')])
            ].index.to_list()
print('Cols to one hot encode:')
print(cat_cols)

Cols to one hot encode:
['Pclass', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']


In [35]:
train_x[cat_cols].isna().any()

Pclass      False
Name        False
Sex         False
Ticket      False
Cabin        True
Embarked     True
dtype: bool

### Pclass, Sex, Ticket

In [36]:
train_x, train_y, test_x = load_data()

In [37]:
cols_to_ohe = ['Pclass', 'Sex', 'Ticket']
all_x = pd.concat([train_x, test_x])
all_x = all_x[['PassengerId'] + cols_to_ohe]
all_x = pd.get_dummies(all_x, columns=cols_to_ohe)

In [38]:
is_in_train = all_x['PassengerId'].isin(train_x['PassengerId'])
is_in_test = all_x['PassengerId'].isin(test_x['PassengerId'])

In [39]:
all_x[is_in_train].to_feather('../features/nn/train_ohe.feather')
all_x[is_in_test].to_feather('../features/nn/test_ohe.feather')

### Cabin, Embarked

In [40]:
train_x, train_y, test_x = load_data()

In [41]:
cols_to_ohe = ['Cabin', 'Embarked']
all_x = pd.concat([train_x, test_x])
all_x = all_x[['PassengerId'] + cols_to_ohe]

In [42]:
all_x['Cabin'] = all_x['Cabin'].apply(
    lambda x: 'Z' if type(x) == 'float' and np.isnan(x) else x
)
all_x['Embarked'] = all_x['Embarked'].apply(
    lambda x: 'Z' if type(x) == 'float' and np.isnan(x) else x
)

In [43]:
all_x = pd.get_dummies(all_x, columns=cols_to_ohe)

In [44]:
is_in_train = all_x['PassengerId'].isin(train_x['PassengerId'])
is_in_test = all_x['PassengerId'].isin(test_x['PassengerId'])

In [45]:
all_x[is_in_train].to_feather('../features/nn/train_ohe_2_z.feather')
all_x[is_in_test].to_feather('../features/nn/test_ohe_2_z.feather')

### Cabin

If exist

In [72]:
train_x, train_y, test_x = load_data()

In [73]:
train_x['Cabin_exist'] = train_x['Cabin'].apply(lambda x: 0 if type(x) == float and np.isnan(x) else 1)
test_x['Cabin_exist'] = test_x['Cabin'].apply(lambda x: 0 if type(x) == float and np.isnan(x) else 1)

In [74]:
train_x[['PassengerId', 'Cabin_exist']].to_feather('../features/nn/cabin_train_exist.feather')
test_x[['PassengerId', 'Cabin_exist']].to_feather('../features/nn/cabin_test_exist.feather')

If in [B, C, D, E, F]

In [78]:
train_x, train_y, test_x = load_data()

In [79]:
train_x['Cabin_B-F'] = train_x['Cabin'].apply(lambda x:
                                                  0 if type(x) == float and np.isnan(x) else
                                                  0 if x[0] not in ['B', 'C', 'D', 'E', 'F'] else
                                                  1
                                             )
test_x['Cabin_B-F'] = train_x['Cabin'].apply(lambda x:
                                                 0 if type(x) == float and np.isnan(x) else
                                                 0 if x[0] not in ['B', 'C', 'D', 'E', 'F'] else
                                                 1
                                            )

In [80]:
train_x[['PassengerId', 'Cabin_B-F']].to_feather('../features/nn/cabin_train_B-F.feather')
test_x[['PassengerId', 'Cabin_B-F']].to_feather('../features/nn/cabin_test_B-F.feather')